In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install bayesian-optimization

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import os
import math
import pickle
import time
import sys

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Lambda, Reshape, Input, Dense, Dropout, Conv1D, Conv1DTranspose, MaxPooling1D, Flatten, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.losses import mse
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from bayes_opt import BayesianOptimization

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


2022-10-01 18:14:32.841045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-01 18:14:32.950845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 18:14:32.950861: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-01 18:14:32.975109: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-01 18:14:33.515243: W tensorflow/stream_executor/pla

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

2022-10-01 18:14:34.079346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-01 18:14:34.115773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 18:14:34.115915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-01 18:14:34.116062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared obj

# BoT-IoT Data Pre-Processing

In [3]:
path = '../botiot/'
all_files = glob.glob(os.path.join(path , '*.csv'))

files_list = []
for file in all_files:
    df = pd.read_csv(file, index_col = None, encoding = 'utf-8', sep = ',', low_memory = False)
    files_list.append(df)

df_traffic = pd.concat(files_list, axis = 0, ignore_index = True)
df_traffic = df_traffic.drop(columns=['pkSeqID', 'stime', 'flgs', 'flgs_number', 'saddr', 'sport', 'daddr', 'dport', 'subcategory'])
df_traffic['proto'] = df_traffic['proto'].map({'tcp': 1, 'arp': 2, 'udp': 3, 'icmp': 4, 'ipv6-icmp': 5})
df_traffic['state'] = df_traffic['state'].map({'REQ': 1, 'RST': 2, 'ACC': 3, 'CON': 4, 'INT': 5, 'URP': 6, 'FIN': 7, 'NRS': 8, 'ECO': 9, 'TST': 10, 'MAS': 11})

In [4]:
# Attacks

df_attack = df_traffic.query('attack == 1')
df_attack = df_attack.drop(columns=['attack'])
df_attack['category'] = df_attack['category'].map({'DDoS': 0, 'DoS': 1, 'Reconnaissance': 2, 'Theft': 3})

df_ddos = df_attack.query('category == 0')
label_ddos = df_ddos.pop('category')

df_dos = df_attack.query('category == 1')
label_dos = df_dos.pop('category')

df_rec = df_attack.query('category == 2')
label_rec = df_rec.pop('category')

df_unknown = df_attack.query('category == 3')
label_unknown = df_unknown.pop('category')

In [5]:
scaler = MinMaxScaler()

df_ddos_norm = scaler.fit_transform(df_ddos)
df_dos_norm = scaler.fit_transform(df_dos)
df_rec_norm = scaler.fit_transform(df_rec)

df_unknown_norm = scaler.fit_transform(df_unknown)

# OpenSet - Classification With Unknown Attack

In [6]:
# Train set

len_ddos_train = int(0.7 * len(df_ddos_norm))
X_ddos_train = df_ddos_norm[:len_ddos_train]

len_dos_train = int(0.7 * len(df_dos_norm))
X_dos_train = df_dos_norm[:len_dos_train]

len_rec_train = int(0.7 * len(df_rec_norm))
X_rec_train = df_rec_norm[:len_rec_train]

np_train = np.concatenate([X_ddos_train, X_dos_train, X_rec_train])
np_label_train = np.concatenate([label_ddos[:len_ddos_train], label_dos[:len_dos_train], label_rec[:len_rec_train]])

df_train = pd.DataFrame(np_train)
df_label_train = pd.DataFrame(np_label_train)

X_train = df_train.to_numpy()
Y_train = df_label_train.to_numpy()

print(len(X_train))

2567575


In [7]:
# Test set

X_ddos_test = df_ddos_norm[len_ddos_train:]
X_dos_test = df_dos_norm[len_dos_train:]
X_rec_test = df_rec_norm[len_rec_train:]
X_unknown_test = df_unknown_norm

np_test = np.concatenate([X_ddos_test, X_dos_test, X_rec_test, X_unknown_test])
np_label_test = np.concatenate([label_ddos[len_ddos_train:], label_dos[len_dos_train:], 
                                 label_rec[len_rec_train:], label_unknown])

df_test = pd.DataFrame(np_test)
df_label_test = pd.DataFrame(np_label_test)

X_test = df_test.to_numpy()
Y_test = df_label_test.to_numpy()

print(len(X_test))

1100470


In [8]:
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

#Y_train = Y_train
#Y_test = Y_test

In [14]:
epochs = 50
learning_rate = 0.0001
batch_size = 30
anomaly_threshold = 0.042

samples, original_dim = X_train.shape
input_shape = (original_dim,)
intermediate_dim = intermediate_dim = int(original_dim / 2)
latent_dim = int(original_dim / 3)

print(latent_dim)

11


In [34]:
def encoder(input_shape, latent_dim, intermediate_dim):
    inputs = Input(shape = input_shape)
    x = layers.Dense(intermediate_dim, activation = 'relu')(inputs)  
    volume_size = K.int_shape(x)
    x = Flatten()(x)
    latent = Dense(latent_dim)(x)
    model = Model(inputs, latent, name = "encoder")
        
    return model, volume_size, inputs

In [35]:
def decoder(input_shape, latent_dim, volume_size):
    latent_inputs = Input(shape = (latent_dim,))
    x = Dense(np.prod(volume_size[1:]))(latent_inputs)
    outputs = Dense(input_shape[0], activation = 'sigmoid')(x)
    model = Model(latent_inputs, outputs, name = 'decoder')
    
    return model

In [36]:
def CNN(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform', input_shape = (input_shape[0], 1)))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    
    return model

In [40]:
opt = optimizers.Nadam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
reduce_lr = ReduceLROnPlateau(moniter = 'val_loss', factor = 0.1, patience = 10)

encoder_model, volume_size, inputs = encoder(input_shape, latent_dim, intermediate_dim)
decoder_model = decoder(input_shape, latent_dim, volume_size)
CNN_model = CNN(input_shape)

model = Model(inputs, decoder_model(CNN_model(encoder_model(inputs))))
model.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy')

ValueError: Input 0 of layer "sequential_4" is incompatible with the layer: expected shape=(None, 35, 1), found shape=(None, 11)

In [ ]:
history = encoder_model.fit(X_train, 
                            Y_train, 
                            epochs = 10, 
                            batch_size = 10,
                            callbacks = [reduce_lr])

In [ ]:
Y_pred = encoder_model.predict(X_train)

In [ ]:
display(pd.DataFrame(Y_pred))